# Google Calendar Event Fetcher

#### The Notebook demonstrates how to programmatically retrieve and process Google Calendar events for a given user and date range.

##### Key Steps:

Authentication: Load user credentials from a token file.

API Call: Fetch events between specified start/end dates using the Google Calendar API.

Data Processing: Extract event details (start/end times, attendees) and structure them into a clean format.

Output: Return a list of events with attendee counts and time slots.
```

### Importing Required Libraries

In [9]:
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

### Defining the retrive_calendar_events Function that includes : 
#### 1. Authenticating with Google Calendar
#### 2. Fetching Events
#### 3. Processing Events & Structuring

In [10]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def retrieve_calendar_events(user, start, end):
    events_list = []
    token_path = "./Keys/" + user.split("@")[0] + ".token"
    
    creds = Credentials.from_authorized_user_file(token_path)
    service = build("calendar", "v3", credentials=creds)
    items = service.events().list(
        calendarId="primary",
        timeMin=start,
        timeMax=end,
        singleEvents=True,
        orderBy="startTime"
    ).execute().get("items", [])
    
    for ev in items:
        atts = [a.get("email", "SELF") for a in ev.get("attendees", [])] or ["SELF"]
        
        if "dateTime" in ev["start"]:
            s = ev["start"]["dateTime"]
        elif "date" in ev["start"]:
            s = ev["start"]["date"] + "T00:00:00+05:30"
        else:
            continue
        
        if "dateTime" in ev["end"]:
            e = ev["end"]["dateTime"]
        elif "date" in ev["end"]:
            e = ev["end"]["date"] + "T00:00:00+05:30"
        else:
            continue
        
        events_list.append({
            "StartTime":    s,
            "EndTime":      e,
            "NumAttendees": len(set(atts)),
            "Attendees":    list(set(atts)),
            "Summary":      ev.get("summary", "")
        })
    
    unique = {}
    for ev in events_list:
        key = (ev["StartTime"], ev["EndTime"], ev["Summary"])
        unique[key] = ev
    deduped = list(unique.values())
    
    def collapse(events):
        events = sorted(events, key=lambda x: x["StartTime"])
        merged = []
        for ev in events:
            if (merged and
                merged[-1]["Summary"] == ev["Summary"] and
                merged[-1]["EndTime"] == ev["StartTime"]):
                merged[-1]["EndTime"] = ev["EndTime"]
            else:
                merged.append(ev.copy())
        return merged
    
    return collapse(deduped)

In [11]:
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "./Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

In [12]:
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "./Keys/" + user.split("@")[0] + ".token"

    try:
        creds = Credentials.from_authorized_user_file(token_path)
        service = build("calendar", "v3", credentials=creds)

        items = service.events().list(
            calendarId='primary',
            timeMin=start,
            timeMax=end,
            singleEvents=True,
            orderBy='startTime'
        ).execute().get('items', [])

        for ev in items:
            # Build attendee list
            attendees = [a.get('email', 'SELF') for a in ev.get('attendees', [])] or ['SELF']

            # Safely extract start_time
            if 'dateTime' in ev['start']:
                start_time = ev['start']['dateTime']
            elif 'date' in ev['start']:
                start_time = ev['start']['date'] + "T00:00:00+05:30"
            else:
                start_time = None

            # Safely extract end_time
            if 'dateTime' in ev['end']:
                end_time = ev['end']['dateTime']
            elif 'date' in ev['end']:
                end_time = ev['end']['date'] + "T00:00:00+05:30"
            else:
                end_time = None

            # Only append if we have at least one timestamp
            if start_time and end_time:
                events_list.append({
                    "StartTime": start_time,
                    "EndTime": end_time,
                    "NumAttendees": len(set(attendees)),
                    "Attendees": list(set(attendees)),
                    "Summary": ev.get("summary", "")
                })
    except Exception as e:
        print(f"Error retrieving calendar events for {user}: {e}")

    return events_list


### Calling ```retrive_calendar_events``` with Auth Token, Start Date & End Date 
#### Date Format : YYYY-MM-DD (T) HH:MM:SS +TIMEZONE (+5:30 Indicates IST Time Zone Asia/Kolkata ) 

In [13]:
events = retrieve_calendar_events(
    "userone.amd@gmail.com",
    '2025-07-17T00:00:00+05:30',
    '2025-07-17T23:59:59+05:30'
)



In [14]:
import os
print(os.path.exists("./Keys/userone.amd.token"))  # Should return True


True


In [15]:
import os
print(os.getcwd())


/home/user


### Output that describing sample event : 

In [16]:
events

[{'StartTime': '2025-07-16T18:00:00+05:30',
  'EndTime': '2025-07-17T09:00:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'},
 {'StartTime': '2025-07-17T09:00:00+05:30',
  'EndTime': '2025-07-17T09:30:00+05:30',
  'NumAttendees': 3,
  'Attendees': ['usertwo.amd@gmail.com',
   'userone.amd@gmail.com',
   'userthree.amd@gmail.com'],
  'Summary': 'Agentic AI Project Status Update'},
 {'StartTime': '2025-07-17T18:00:00+05:30',
  'EndTime': '2025-07-18T09:00:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'}]